# Generate Market Book by Order (MBO) quotes using dedicated service - C#

### Overview
This sample is designed to request quotes and illustrate full order book messaging.

The Market Book by Order (MBO) describes an order-based (or quote-based) data feed that provides the ability to view individual queue position, the full order book (all the depths) and the details of each individual order or quote at each price level.

**Note : This sample is not about Market Book by Limit (MBL). For the MBL please refer the notebook [Tick] bbo10_and_trades**

### Inputs/outputs
Quotes extraction sample requires instrument's identifier, date time intervals as per inputs and returns the occured quotes information.

### Services used
This sample uses *gRPC requests* in order to retrieve quotes information from the hosted service. The queried endpoint in this script are:
* *TickQuotesService*: to directly retrieve quotes data from the server.

### Packages required
1. Systemathics:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source:
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    
***

# Run Quotes sample

### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages Systemathics.Apis, 0.13.2-pre

In [2]:
using System.IO;

using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;

using TickQuotesService = Systemathics.Apis.Services.Tick.V1.TickQuotesService;
using TickQuotesRequest = Systemathics.Apis.Services.Tick.V1.TickQuotesRequest;
using TickQuotesResponse = Systemathics.Apis.Services.Tick.V1.TickQuotesResponse;
using Quote = Systemathics.Apis.Type.Shared.V1.Quote;


### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
var headers = new Grpc.Core.Metadata();
try { headers = TokenHelpers.GetTokenAsMetaData(); } catch {}

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *tick quotes* service, we need to specify:
* Instrument identifier : the ICE code and the source
* Time period selection: select start and end dates and time intervals (Using UTC Time Zone)
* Tick trades request parameters

#### 3.1 Instrument selection

In [4]:
// Tha data is provided by ICE : let's use the ICE mapping codes to generate the identifier
// The ICE ticker
var ticker = "E:BNP";
// The source for full order book : Euronext
var source = "EQUITY_L2_973";

#### 3.2 Time period delimitation

In [5]:
// Create time intervals
// Full order book data avaialble (sample) : from 2021-11-01 to 2021-11-12
var start = new DateTime(2021, 11, 01);
var end = new DateTime(2021, 11, 01);
var dateIntervals = new DateInterval()
{
  StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
  EndDate = new Date { Year = end.Year, Month = end.Month, Day = end.Day }
};

// Build the tick quotes request time interval (we are using Google date time format)
// UTC time zone
var timeInterval = new TimeInterval()
{
 StartTime = new TimeOfDay { Hours = 12, Minutes = 00, Seconds = 00 },
 EndTime = new TimeOfDay { Hours = 20, Minutes = 30, Seconds = 00 } 
};

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [6]:
// Generate constraints based on the previous time selection
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

The following code snippets create requests for *quotes* and instantiate the service:

In [7]:
// Generate the tick quotes request
var request = new TickQuotesRequest { Constraints = constraints };
var identifier = new Identifier { Exchange = source, Ticker = ticker};
request.Identifiers.Add(new [] {identifier}); 

In [8]:
// Instantiate the tick quotes service client
var service = new TickQuotesService.TickQuotesServiceClient(channel);

### Step 4: Visualize data

#### 4.1 Retrieve quotes data

The following code exports quotes to some file

In [9]:
// Get the quotes and export data into csv file
var call = service.TickQuotes(request, headers);
var d = new DirectoryInfo("output"); if (!d.Exists) { d.Create(); }
using(var sw = File.CreateText($"output/{ticker}_quotes.csv"))
await foreach (var current in call.ResponseStream.ReadAllAsync())
{
    if (current.Mapping != null)
    {
        sw.WriteLine("TimeStamp,Mapping,ID,Side,Price,Size,Condition");
        foreach(var a in current.Mapping.Values)
        {
            var i = a.Identifier;
            var m = a.Value;
            sw.WriteLine($",{m}=>{i.Provider}/{i.Exchange}/{i.Ticker},,,,");
        }

        continue;
    }

    if (current.Data == null)
    {
        continue;
    }
    
    // Quotes
    var ts = current.Data.TimeStamp;
    var mapping = current.Data.Mapping;
    foreach (var quote in current.Data.Quotes) 
    {      
        sw.WriteLine($"{ts},{mapping},{quote.Id},{quote.Side},{quote.Price},{quote.Size},{quote.Condition}");
    }
}